In [1]:
%load_ext autoreload
%autoreload 2

This is a notebook that illustrates how to run the end-to-end (e2e) workflow. Determining suitable parameters for the workflow are for operational considerations are explained in the ops library [`dist-s1-enumerator`](https://github.com/opera-adt/dist-s1-enumerator) - specifically, see this [notebook](https://github.com/opera-adt/dist-s1-enumerator/blob/dev/notebooks/A__Staging_Inputs_for_One_MGRS_Tile.ipynb).

In [6]:
from pathlib import Path

from dist_s1.workflows import (
    run_burst_disturbance_workflow,
    run_dist_s1_localization_workflow,
    run_dist_s1_workflow,
    run_disturbance_merge_workflow,
    run_dist_s1_packaging_workflow
)

In [3]:
## Example 0

# mgrs_tile_id = '10SGD'
# post_date = '2025-01-02'
# track_number = 137
# dst_dir = Path('out')
# memory_strategy = 'high'


## Example 1 - Los Angeles Wildfire
mgrs_tile_id = '11SLT'
post_date = '2025-01-21'
track_number = 71
dst_dir = Path('los-angeles')
memory_strategy = 'high'
lookback_strategy='immediate_lookback'

This takes some time as it not only generates the runconfig but also localizes the necessary data. It will not overwrite files that have been previously downloaded.

Few additional notes:

1. The runconfig can be serialized to yml and then read from that file.
2. The runconfig manages all the paths for the workflows from the initial set of inputs. It's data model is messy and can be confusing. But that's the workhorse of this library.

In [18]:
run_config = run_dist_s1_localization_workflow(
    mgrs_tile_id,
    post_date,
    track_number,
    post_date_buffer_days=1,
    dst_dir=dst_dir,
    input_data_dir=dst_dir,
    lookback_strategy=lookback_strategy,
)



Using immediate lookback strategy


In [15]:
#run_config.confirmation_strategy='compute_baseline'

In [19]:
#run_burst_disturbance_workflow(run_config)
# run_disturbance_merge_workflow(run_config)
run_dist_s1_packaging_workflow(run_config)

Using previous product for confirmation
Running disturbance confirmation
No previous product found for confirmation. Assuming this is the first product.
This is a first product tile with date 2025-01-21 13:52:46
Packaging CONF DB disturbance tifs
Exporting /Users/cmarshak/bekaert-team/dist-s1/notebooks/los-angeles/OPERA_L3_DIST-ALERT-S1_T11SLT_20250121T135246Z_20250606T160028Z_S1_30_v0.1/OPERA_L3_DIST-ALERT-S1_T11SLT_20250121T135246Z_20250606T160028Z_S1_30_v0.1_GEN-DIST-STATUS.tif
Exporting /Users/cmarshak/bekaert-team/dist-s1/notebooks/los-angeles/OPERA_L3_DIST-ALERT-S1_T11SLT_20250121T135246Z_20250606T160028Z_S1_30_v0.1/OPERA_L3_DIST-ALERT-S1_T11SLT_20250121T135246Z_20250606T160028Z_S1_30_v0.1_GEN-DIST-STATUS-ACQ.tif
Exporting /Users/cmarshak/bekaert-team/dist-s1/notebooks/los-angeles/OPERA_L3_DIST-ALERT-S1_T11SLT_20250121T135246Z_20250606T160028Z_S1_30_v0.1/OPERA_L3_DIST-ALERT-S1_T11SLT_20250121T135246Z_20250606T160028Z_S1_30_v0.1_GEN-METRIC-MAX.tif
Exporting /Users/cmarshak/bekaert

In [8]:
# run_config

RunConfigData(pre_rtc_copol=[PosixPath('los-angeles/11SLT/71/2024-08-30/OPERA_L2_RTC-S1_T071-151226-IW2_20240830T135251Z_20240831T035515Z_S1A_30_v1.0_VV.tif'), PosixPath('los-angeles/11SLT/71/2024-09-11/OPERA_L2_RTC-S1_T071-151226-IW2_20240911T135251Z_20241011T172143Z_S1A_30_v1.0_VV.tif'), PosixPath('los-angeles/11SLT/71/2024-09-23/OPERA_L2_RTC-S1_T071-151226-IW2_20240923T135252Z_20240923T191619Z_S1A_30_v1.0_VV.tif'), PosixPath('los-angeles/11SLT/71/2024-10-05/OPERA_L2_RTC-S1_T071-151226-IW2_20241005T135252Z_20241005T173141Z_S1A_30_v1.0_VV.tif'), PosixPath('los-angeles/11SLT/71/2024-10-17/OPERA_L2_RTC-S1_T071-151226-IW2_20241017T135252Z_20241017T174543Z_S1A_30_v1.0_VV.tif'), PosixPath('los-angeles/11SLT/71/2024-10-29/OPERA_L2_RTC-S1_T071-151226-IW2_20241029T135252Z_20241029T233207Z_S1A_30_v1.0_VV.tif'), PosixPath('los-angeles/11SLT/71/2024-11-10/OPERA_L2_RTC-S1_T071-151226-IW2_20241110T135252Z_20241110T175010Z_S1A_30_v1.0_VV.tif'), PosixPath('los-angeles/11SLT/71/2024-11-22/OPERA_L2_RT

In [10]:
(run_config.water_mask_path).exists()

True